#### Goal for this webscrape phase: "go into" a page with links to "zoom in"/loop through links that then take you to another level that will then loop through links to scrape each webpage with in
    Challenge:
    -there are 14 styles (categories) and around ~120 links (subcategories)
    -each style can have multiptle pages to loop through

###### Starting small by scraping one page

In [5]:
import cloudscraper
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd  
import time 
from random import randint


In [ ]:
url = "https://www.beeradvocate.com/beer/profile/143/1352/" # this was first test page scraped
# url ="https://www.beeradvocate.com/beer/profile/17033/53930/" #this one does not have ratings
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
scraper = cloudscraper.create_scraper() 
info = scraper.get(url) 
print(info.status_code) 
 

In [ ]:
soup = BeautifulSoup(info.text, 'lxml') 

In [ ]:
name=soup.h1.find(string=True)
name

In [ ]:
soup.find('h1').text
#will give me both the beer title and brewery merged together

In [ ]:
# soup.find('dl',{'class':'beerstats'}).text

In [ ]:
brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
brewery

In [ ]:
loaction=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
loaction

In [ ]:
location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
location2

In [ ]:
style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
style

In [ ]:
ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
ba_score
##official score on ba site 74::'okay'

In [ ]:
over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
over_all_rank
##over all rank is against all beers in ba site

In [ ]:
style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
style_rank
## style rank is among all beers within style category

In [ ]:
alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
alcohol_percent

In [ ]:
avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
avg_rating
# Average across all ratings for this beer

In [ ]:
percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
percent_dev
# Percentage of deviation within the ratings

In [ ]:
review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
review_count
# Number of reviews for this beer

In [ ]:
rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
rating_count
# Number of ratings for this beer

In [ ]:
status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
status
# the beer's current activity status

In [ ]:
date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
date_added
# The date this beer was added to the site

###### zooming out one level to scrape through "bocks" pg 1 with 50 results

In [ ]:
# youtube page: John Watson Rooney helped get more of the wabscraping process 
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=0'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website) 
print(info.status_code)


In [ ]:
soup = BeautifulSoup(info.text, 'lxml')

In [ ]:
table=soup.find('table')
# table is where the links are concentrated in 

In [ ]:
table.find_all("a", href=re.compile("profile"))[::2]

In [ ]:
links=[link['href'] for link in table.find_all("a", href=re.compile("profile"))[::2]]
print(links)

In [ ]:
beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])


In [ ]:
beerlinks


##### Building together code to loop through links on one page  

In [ ]:
# print(f'{variable im using in the for loop} : {variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')
# https://medium.com/swlh/improve-your-web-scraper-with-limited-retry-loops-python-35e21730cbf5
# https://www.youtube.com/watch?v=sIOMDu6MXJQ&list=PLRzwgpycm-Fio7EyivRKOBN4D3tfQ_rpu&index=3
# https://www.youtube.com/watch?v=8dTpNajxaH0&t=503s
# https://www.freecodecamp.org/news/how-to-scrape-multiple-web-pages-using-python/
# https://medium.com/@udayshadani/errors-we-can-get-during-web-scraping-and-how-to-handle-it-41de0c2f6ec3#:~:text=Connection%20errors%3A%20These%20errors%20occur,a%20firewall%20blocking%20the%20connection.

In [ ]:
# testlink='https://www.beeradvocate.com/beer/profile/143/1352/'
# for page pagination, ran a range to an imposiblly high number, by 50  (results per page that url uses to fill start param)
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=100'
scraper = cloudscraper.create_scraper() 
response = scraper.get(website)  
soup = BeautifulSoup(response.text, 'lxml')
table=soup.find('table')
# print(table)
beerlist=[]

beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])
# print(beerlinks)

for link in beerlinks:
    scraper = cloudscraper.create_scraper()
    r=scraper.get(link, headers=header)
    soup=BeautifulSoup(r.text,'lxml')
    name=soup.h1.find(string=True)
    try:    
        brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
    except:
        brewery= 'Na'
    try:
        location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
    except:
        location= 'Na'
    try:
        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
    except:
        location2= 'Na'
    try:
        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
    except:
        style= 'Na'
    try:
        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
    except:
        ba_score= 'Na'
    try:
        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
    except:
        over_all_rank= 'Na'
    try:
        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
    except:
        style_rank= 'Na'
    try:
        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
    except:
        alcohol_percent= 'Na'
    try:
        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
    except:
        avg_rating= 'Na'
    try:
        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
    except:
        percent_dev= 'Na'
    try:
        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
    except:
        review_count= 'Na'
    try:
        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
    except:
        rating_count= 'Na'
    try:
        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
    except:
        status= 'Na'
    try:
        date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
    except:
        date_added= 'Na'
    beer_info= {
                'name': name,
                'brewery': brewery,
                'location':location,
                'location2':location2,
                'style': style,
                'ba_score':ba_score,
                'over_all_rank': over_all_rank,
                'style_rank': style_rank,
                'alcohol_percent': alcohol_percent,
                'avg_rating':avg_rating,
                'percent_dev': percent_dev,
                'review_count': review_count,
                'rating_count':rating_count,
                'status': status,
                'date_added': date_added
                }
    beerlist.append(beer_info)
    print(f'{link}-{name}')

# print({variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')

In [ ]:
print(beerlist)

In [ ]:
df=pd.DataFrame(beerlist)
df

In [ ]:
len(beerlinks)

##### Building together code to loop through links of different styles of beer (zooming out another level/ another loop)
-challanges: url for different beer styles has a slight change in on of the parameters for each style of beer, need to be mindful of this in loop, meaing a few changes must be implimented on code above


In [ ]:
# root='https://www.beeradvocate.com'
# header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

    
# website=f'{root}/beer/styles/'
# scraper = cloudscraper.create_scraper() 
# info = scraper.get(website)  
# soup = BeautifulSoup(info.text, 'lxml')
# container=soup.find('div',{'id':'ba-content'})
# container.find_all('a', href=True)[0:-1]

# style_links=[]
# for link in container.find_all('a', href=True)[0:-1]:
#     style_links.append(link['href'])

#  beerlinks=[]
#         for link in table.find_all("a", href=re.compile("profile"))[::2]:
#             beerlinks.append(root + link['href'])

In [ ]:
# style_links

In [ ]:
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

website=f'{root}/beer/styles/'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website)  
soup = BeautifulSoup(info.text, 'lxml')
container=soup.find('div',{'id':'ba-content'})

beerlist=[]

style_href=[]
for link in container.find_all('a', href=True)[0:-1]:
    style_href.append(link['href'])

for h in style_href:
    webber=f'{root}{h}'
    time.sleep(5)
    print(webber)
    

#     for x in range(0,100000,50):
#         print(requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}'))
#         if requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}').status_code == 200:    
#             web=f'{webber}?sort=revsD&start={x}'
#             scraper = cloudscraper.create_scraper() 
#             info = scraper.get(web)  
#             soup = BeautifulSoup(info.text, 'lxml')
#             table=soup.find('table')

#             beerlinks=[]
#             for link in table.find_all("a", href=re.compile("profile"))[::2]:
#                 beerlinks.append(root + link['href'])


#             for link in beerlinks:
#                 r=requests.get(link, headers=header)
#                 soup=BeautifulSoup(r.content,'lxml')
#                 name=soup.h1.find(string=True)
#                 try:    
#                     brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
#                 except:
#                     brewery= 'Na'
#                 try:
#                     location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
#                 except:
#                     location= 'Na'
#                 try:
#                     location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
#                 except:
#                     location2= 'Na'
#                 try:
#                     style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     style= 'Na'
#                 try:
#                     ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     ba_score= 'Na'
#                 try:
#                     over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     over_all_rank= 'Na'
#                 try:
#                     style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
#                 except:
#                     style_rank= 'Na'
#                 try:
#                     alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
#                 except:
#                     alcohol_percent= 'Na'
#                 try:
#                     avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     avg_rating= 'Na'
#                 try:
#                     percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
#                 except:
#                     percent_dev= 'Na'
#                 try:
#                     review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
#                 except:
#                     review_count= 'Na'
#                 try:
#                     rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
#                 except:
#                     rating_count= 'Na'
#                 try:
#                     status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
#                 except:
#                     status= 'Na'
#                 try:
#                     date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
#                 except:
#                     date_added= 'Na'
#                 beer_info= {
#                              'name': name,
#                              'brewery': brewery,
#                             'location':location,
#                             'location2':location2,
#                             'style': style,
#                             'ba_score':ba_score,
#                             'over_all_rank': over_all_rank,
#                             'style_rank': style_rank,
#                             'alcohol_percent': alcohol_percent,
#                             'avg_rating':avg_rating,
#                             'percent_dev': percent_dev,
#                             'review_count': review_count,
#                             'rating_count':rating_count,
#                             'status': status,
#                             'date_added': date_added
#                             }
#                 beerlist.append(beer_info)
#                 print(f'{h}-{x}-{link}-{name}')
#         else:
#             break          



In [ ]:
# root='https://www.beeradvocate.com'  #base url
# headers= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
# website=f'{root}/beer/styles/'  #base url + path to beer style list 
# scraper = cloudscraper.create_scraper() #calling a scraper session
# response = scraper.get(website,headers=headers)  #built scraper (documnetation says treat it like response)
# soup = BeautifulSoup(response.text, 'html.parser')  #soup of beer style list 
# container=soup.find('div',{'id':'ba-content'}) #getting section of html that contains most of href style ids'
# # print(response.status_code)    ##ran and got 200 scraper works

# beerlist=[]

style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #getting text of href that needs to later be paired with root and other parameters
# print(len(style_href)) ## 120 links from beer style list
for h in style_href: #contatinating root and h to pull up style lists
#     webber=f'{root}{h}'
#     print(f'{webber}')
for x in range(0,100000,50):
    m=f'{root}{h}?sort=revsD&start={x}'
    print(m)
#     scraper = cloudscraper.create_scraper()
    res = scraper.get(webber,headers=headers)
    time.sleep(3)
    print(scraper.get(f'{res}').status_code)
    print(response.status_code) ##ran 200 for each response from webber
    if scraper.get(webber).status_code == 200:   
        scraper = cloudscraper.create_scraper()
        response = scraper.get(webber,headers=headers)
#         time.sleep(3)
#         print(response.status_code)
        print(f'{webber}')
        for x in range(0,100000,50):
            web=f'{webber}{x}'
            if scraper.get(f'{webber}{x}').status_code == 200:        
                scraper = cloudscraper.create_scraper() 
                info = scraper.get(website)  
                soup = BeautifulSoup(info.text, 'lxml')
                table=soup.find('table')
                beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
                for link in beerlinks:
                    scraper = cloudscraper.create_scraper()
                    r=scraper.get(link, headers=header)
                    soup=BeautifulSoup(r.text,'lxml')
                    name=soup.h1.find(string=True)
                    try:    
                        brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
                    except:
                        brewery= 'Na'
                    try:
                        location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
                    except:
                        location= 'Na'
                    try:
                        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
                    except:
                        location2= 'Na'
                    try:
                        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
                    except:
                        style= 'Na'
                    try:
                        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
                    except:
                        ba_score= 'Na'
                    try:
                        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
                    except:
                        over_all_rank= 'Na'
                    try:
                        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
                    except:
                        style_rank= 'Na'
                    try:
                        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
                    except:
                        alcohol_percent= 'Na'
                    try:
                        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
                    except:
                        avg_rating= 'Na'
                    try:
                        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
                    except:
                        percent_dev= 'Na'
                    try:
                        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
                    except:
                        review_count= 'Na'
                    try:
                        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
                    except:
                        rating_count= 'Na'
                    try:
                        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
                    except:
                        status= 'Na'
                    try:
                        date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
                    except:
                        date_added= 'Na'
                    beer_info= {
                                'name': name,
                                 'brewery': brewery,
                                'location':location,
                                'location2':location2,
                                'style': style,
                                'ba_score':ba_score,
                                'over_all_rank': over_all_rank,
                                'style_rank': style_rank,
                                'alcohol_percent': alcohol_percent,
                                'avg_rating':avg_rating,
                                'percent_dev': percent_dev,
                                'review_count': review_count,
                                'rating_count':rating_count,
                                'status': status,
                                'date_added': date_added
                                }
                    beerlist.append(beer_info)
                    print(f'{h}-{x}-{link}-{name}')
            else:
                break 


In [27]:
def cook_soup(url):
    scraper = cloudscraper.create_scraper()
    headers= {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
    response = scraper.get(url,headers=headers)
    soup = BeautifulSoup(response.text,'lxml')
    print(f'url: {url} - status:{response.status_code}-')
    return soup 

#will create an instance of cloudscraper session for url 
# https://stackoverflow.com/questions/73027049/how-to-ensure-response-200-for-multiple-requests-when-webscraping

In [28]:
test=cook_soup('https://www.beeradvocate.com/beer/profile/57/7714/')


url: https://www.beeradvocate.com/beer/profile/57/7714/ - status:200 -


In [29]:
url="https://www.beeradvocate.com/beer/profile/417/47795/"
test_2=cook_soup(url)

url: https://www.beeradvocate.com/beer/profile/417/47795/ - status:200 -


In [40]:

beerlist=[]
root='https://www.beeradvocate.com' # base url
website=f'{root}/beer/styles/' #url that has links to styles of beers (14 styles, and 120 links)
soup=cook_soup(website) #makes soup of previous url
container=soup.find('div',{'id':'ba-content'}) #looks for place where href are
style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #saves hrefs
for h in style_href:
    webber=f'{root}{h}'
#     print(webber)
    scraper(webber)
    for x in range(0,100000,50):
        web=f'{root}{h}?sort=revsD&start={x}'
        soup=cook_soup(web)
        time.sleep(3)
#         if response.status_code != 200:
#             break ## issue is that it keeps going after last results page with existing page with empty results
#         else:
        table=soup.find('table')
        beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
        print(beerlinks)
        if len(beerlinks) == 0:
            break
        else:        
            for link in beerlinks:
                soup=cook_soup(link)
                name=soup.h1.find(string=True)
                try:    
                    brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
                except:
                    brewery= 'Na'
                try:
                    location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
                except:
                    location= 'Na'
                try:
                        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
                except:
                        location2= 'Na'
                try:
                        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
                except:
                        style= 'Na'
                try:
                        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
                except:
                        ba_score= 'Na'
                try:
                        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
                except:
                        over_all_rank= 'Na'
                try:
                        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
                except:
                        style_rank= 'Na'
                try:
                        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
                except:
                        alcohol_percent= 'Na'
                try:
                        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
                except:
                        avg_rating= 'Na'
                try:
                        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
                except:
                        percent_dev= 'Na'
                try:
                        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
                except:
                        review_count= 'Na'
                try:
                        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
                except:
                        rating_count= 'Na'
                try:
                        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
                except:
                    status= 'Na'
                try:
                    date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
                except:
                    date_added= 'Na'
                beer_info= {
                            'name': name,
                            'brewery': brewery,
                            'location':location,
                            'location2':location2,
                            'style': style,
                            'ba_score':ba_score,
                            'over_all_rank': over_all_rank,
                            'style_rank': style_rank,
                            'alcohol_percent': alcohol_percent,
                            'avg_rating':avg_rating,
                            'percent_dev': percent_dev,
                            'review_count': review_count,
                            'rating_count':rating_count,
                            'status': status,
                            'date_added': date_added
                            }
                beerlist.append(beer_info)
                print(f'{h}-{x}-{link}-{name}')    
    
        

url: https://www.beeradvocate.com/beer/styles/ - status:200-
url: https://www.beeradvocate.com/beer/styles/32/ - status:200 -
url: https://www.beeradvocate.com/beer/styles/32/?sort=revsD&start=0 - status:200-
['https://www.beeradvocate.com/beer/profile/143/1352/', 'https://www.beeradvocate.com/beer/profile/35/14309/', 'https://www.beeradvocate.com/beer/profile/29/1252/', 'https://www.beeradvocate.com/beer/profile/35/68000/', 'https://www.beeradvocate.com/beer/profile/590/2565/', 'https://www.beeradvocate.com/beer/profile/35/74579/', 'https://www.beeradvocate.com/beer/profile/590/62671/', 'https://www.beeradvocate.com/beer/profile/125/2272/', 'https://www.beeradvocate.com/beer/profile/337/1070/', 'https://www.beeradvocate.com/beer/profile/2137/3835/', 'https://www.beeradvocate.com/beer/profile/302/41422/', 'https://www.beeradvocate.com/beer/profile/66/1873/', 'https://www.beeradvocate.com/beer/profile/184/20299/', 'https://www.beeradvocate.com/beer/profile/35/520715/', 'https://www.beer

url: https://www.beeradvocate.com/beer/profile/31272/171485/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/31272/171485/-Bockslider Toadies Texas Bock
url: https://www.beeradvocate.com/beer/profile/38015/170131/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/38015/170131/-Love Honey
url: https://www.beeradvocate.com/beer/profile/7734/76116/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/7734/76116/-Nero's 1st Century Double Dark Malt Ale
url: https://www.beeradvocate.com/beer/profile/26762/401984/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/26762/401984/-Crawford Bock
url: https://www.beeradvocate.com/beer/profile/514/11985/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/514/11985/-Bokbier
url: https://www.beeradvocate.com/beer/profile/46029/453249/ - status:200-
/beer/styles/32/-0-https://www.beeradvocate.com/beer/profile/46029/453249/-Der Sch

url: https://www.beeradvocate.com/beer/profile/45456/445767/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/45456/445767/-Funeral Bock
url: https://www.beeradvocate.com/beer/profile/1888/64328/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/1888/64328/-Winter Lager
url: https://www.beeradvocate.com/beer/profile/2431/350692/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/2431/350692/-Steam Brew German Red
url: https://www.beeradvocate.com/beer/profile/530/57623/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/530/57623/-Hertog Jan Lentebock
url: https://www.beeradvocate.com/beer/profile/783/8416/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/783/8416/-Faxe Fest Bock
url: https://www.beeradvocate.com/beer/profile/22072/67318/ - status:200-
/beer/styles/32/-50-https://www.beeradvocate.com/beer/profile/22072/67318/-Baphomet
url: https://www.beerad

url: https://www.beeradvocate.com/beer/profile/16271/41963/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/16271/41963/-Bock
url: https://www.beeradvocate.com/beer/profile/6627/20992/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/6627/20992/-Pfungstädter St. Nikolaus
url: https://www.beeradvocate.com/beer/profile/24571/90388/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/24571/90388/-San Martino La Gold
url: https://www.beeradvocate.com/beer/profile/2304/21646/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/2304/21646/-Bibock
url: https://www.beeradvocate.com/beer/profile/6844/182259/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/6844/182259/-Bourbon Barrel Bock
url: https://www.beeradvocate.com/beer/profile/1252/48457/ - status:200-
/beer/styles/32/-100-https://www.beeradvocate.com/beer/profile/1252/48457/-Quilmes Bock
url: https://ww

url: https://www.beeradvocate.com/beer/styles/32/?sort=revsD&start=150 - status:200-
['https://www.beeradvocate.com/beer/profile/943/3029/', 'https://www.beeradvocate.com/beer/profile/9076/241149/', 'https://www.beeradvocate.com/beer/profile/507/19207/', 'https://www.beeradvocate.com/beer/profile/29666/577830/', 'https://www.beeradvocate.com/beer/profile/6063/148526/', 'https://www.beeradvocate.com/beer/profile/628/103996/', 'https://www.beeradvocate.com/beer/profile/458/636737/', 'https://www.beeradvocate.com/beer/profile/661/321873/', 'https://www.beeradvocate.com/beer/profile/18412/198241/', 'https://www.beeradvocate.com/beer/profile/29171/112363/', 'https://www.beeradvocate.com/beer/profile/22243/145503/', 'https://www.beeradvocate.com/beer/profile/30949/589413/', 'https://www.beeradvocate.com/beer/profile/607/130408/', 'https://www.beeradvocate.com/beer/profile/498/144756/', 'https://www.beeradvocate.com/beer/profile/21200/79334/', 'https://www.beeradvocate.com/beer/profile/544/45

...

url: https://www.beeradvocate.com/beer/profile/7628/119171/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/7628/119171/-The Monarchy Methusalem
url: https://www.beeradvocate.com/beer/profile/9826/131221/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/9826/131221/-ALTitude Altbier
url: https://www.beeradvocate.com/beer/profile/1949/133793/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/1949/133793/-Alt
url: https://www.beeradvocate.com/beer/profile/19012/90037/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/19012/90037/-Appalachia Alt
url: https://www.beeradvocate.com/beer/profile/35261/304565/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/35261/304565/-Altbier
url: https://www.beeradvocate.com/beer/profile/34069/161985/ - status:200-
/beer/styles/86/-200-https://www.beeradvocate.com/beer/profile/34069/161985/-Doppel Alt
url: https://www.

url: https://www.beeradvocate.com/beer/profile/1177/200556/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/1177/200556/-Northern Gale Sticke
url: https://www.beeradvocate.com/beer/profile/125/140398/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/125/140398/-Doppelsticke Altbier
url: https://www.beeradvocate.com/beer/profile/54017/633980/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/54017/633980/-1845 Sticke
url: https://www.beeradvocate.com/beer/profile/54017/622208/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/54017/622208/-1845
url: https://www.beeradvocate.com/beer/profile/2652/75400/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/2652/75400/-Latzenbier
url: https://www.beeradvocate.com/beer/profile/43740/209559/ - status:200-
/beer/styles/86/-250-https://www.beeradvocate.com/beer/profile/43740/209559/-Altbier
url: https://www.bee

url: https://www.beeradvocate.com/beer/styles/86/?sort=revsD&start=300 - status:200-
['https://www.beeradvocate.com/beer/profile/587/49083/', 'https://www.beeradvocate.com/beer/profile/2555/44998/', 'https://www.beeradvocate.com/beer/profile/54216/375757/', 'https://www.beeradvocate.com/beer/profile/60416/590293/', 'https://www.beeradvocate.com/beer/profile/22723/597987/', 'https://www.beeradvocate.com/beer/profile/24822/71141/', 'https://www.beeradvocate.com/beer/profile/51154/645096/', 'https://www.beeradvocate.com/beer/profile/34520/120822/', 'https://www.beeradvocate.com/beer/profile/45307/396291/', 'https://www.beeradvocate.com/beer/profile/43537/224771/', 'https://www.beeradvocate.com/beer/profile/35017/123655/', 'https://www.beeradvocate.com/beer/profile/125/42249/', 'https://www.beeradvocate.com/beer/profile/30247/205836/', 'https://www.beeradvocate.com/beer/profile/35646/141591/', 'https://www.beeradvocate.com/beer/profile/2097/45079/', 'https://www.beeradvocate.com/beer/profi

url: https://www.beeradvocate.com/beer/profile/52604/349531/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/52604/349531/-Olde Town Alt
url: https://www.beeradvocate.com/beer/profile/51814/626786/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/51814/626786/-Altbier
url: https://www.beeradvocate.com/beer/profile/34069/308579/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/34069/308579/-Der Hammer
url: https://www.beeradvocate.com/beer/profile/9408/598372/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/9408/598372/-Old Dusseldorf
url: https://www.beeradvocate.com/beer/profile/545/111464/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/545/111464/-Venloosch Alt
url: https://www.beeradvocate.com/beer/profile/1432/154230/ - status:200-
/beer/styles/86/-300-https://www.beeradvocate.com/beer/profile/1432/154230/-Old Soul
url: https://www.beeradvo

ConnectionError: HTTPSConnectionPool(host='www.beeradvocate.com', port=443): Max retries exceeded with url: /beer/profile/603/101544/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3552f1790>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [46]:
df=pd.DataFrame(beerlist)
df

,name,brewery,location,location2,style,ba_score,over_all_rank,style_rank,alcohol_percent,avg_rating,percent_dev,review_count,rating_count,status,date_added
0,Shiner Bock,Spoetzl Brewery,Texas,United States,Bock,74,"Ranked #28,000",Ranked #96,4.5%,3.21,19.63%,"1,234","4,564",Active,"Oct 03, 1999"
1,Samuel Adams Chocolate Bock,Boston Beer Company (Samuel Adams),Massachusetts,United States,Bock,82,"Ranked #24,951",Ranked #70,5.8%,3.64,16.21%,"1,247","3,144",Active,"Nov 20, 2003"
2,Michelob AmberBock,Anheuser-Busch,Missouri,United States,Bock,68,"Ranked #28,503",Ranked #100,5.1%,2.91,22.34%,732,"1,970",Active,"Aug 17, 2001"
3,Samuel Adams Cherry Chocolate Bock,Boston Beer Company (Samuel Adams),Massachusetts,United States,Bock,79,"Ranked #26,931",Ranked #85,5.8%,3.47,19.6%,188,875,Active,"Apr 11, 2011"
4,Uff-da,New Glarus Brewing Company,Wisconsin,United States,Bock,86,"Ranked #19,038",Ranked #27,6.7%,3.83,10.97%,342,663,Active,"Jan 16, 2002"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6579,Whatley's Alt,Altbier,Colorado,United States,Na,Avg:,Na,Na,5%,5.46%,Reviews:,Ratings:,Status:,Rated:,Wants:
6580,Doppel Alt,Altbier,Germany,Na,Na,Avg:,Na,Na,7%,2.94%,Reviews:,Ratings:,Status:,Rated:,Wants:
6581,Graves Landing,Altbier,Massachusetts,United States,Na,Avg:,Na,Na,4.8%,4.15%,Reviews:,Ratings:,Status:,Rated:,Wants:
6582,RSTYMTL,Altbier,New Hampshire,United States,Na,Avg:,Na,Na,6%,8.57%,Reviews:,Ratings:,Status:,Rated:,Wants:


In [49]:
df['style'].unique()

array(['Bock', 'Na', 'Doppelbock', 'Eisbock', 'Maibock', 'Weizenbock',
       'Altbier'], dtype=object)